In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import tensorflow as tf
from tensorflow.keras import layers
import seaborn as sns

pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

2022-12-31 16:30:35.453146: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (None)/charset_normalizer (3.0.1) doesn't match a supported version!
  warnings.warn(


1 Physical GPUs, 1 Logical GPUs


2022-12-31 16:30:36.544057: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-31 16:30:36.555594: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-31 16:30:36.555725: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-31 16:30:36.556230: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other oper

In [2]:
USE_BT = True


DATASET_ROOT="data/"
TRAIN_LOCATIONS_FILE="SignatureLocs_altered.csv"
GENERATED_TRAIN_LOCATIONS_FILE="SignatureLocs_altered_generated.csv"
TEST_LOCATIONS_FILE="TestLocs_altered.csv"
TRAIN_STRENGTHS_FILE="P_Signatures.csv" if USE_BT else "P_SA_Signatures.csv"
GENERATED_TRAIN_STRENGTHS_FILE="P_Signatures_generated.csv" if USE_BT else "P_SA_Signatures_generated.csv"
TEST_STRENGTHS_FILE="P_Tests.csv" if USE_BT else "P_SA_Tests.csv"
NUMBER_OF_BEACONS=57 if USE_BT else 11

In [3]:
df_train_strengths = pd.read_csv(DATASET_ROOT+TRAIN_STRENGTHS_FILE, sep=';', names=[x for x in range(NUMBER_OF_BEACONS)])
df_train_locs = pd.read_csv(DATASET_ROOT+TRAIN_LOCATIONS_FILE, sep=';', names=['x','y'], dtype=float)
df_generated_train_strengths = pd.read_csv(DATASET_ROOT+TRAIN_STRENGTHS_FILE, sep=';', names=[x for x in range(NUMBER_OF_BEACONS)])
df_generated_train_locs = pd.read_csv(DATASET_ROOT+TRAIN_LOCATIONS_FILE, sep=';', names=['x','y'], dtype=float)
df_test_strengths = pd.read_csv(DATASET_ROOT+TEST_STRENGTHS_FILE, sep=';', names=[x for x in range(NUMBER_OF_BEACONS)])
df_test_locs = pd.read_csv(DATASET_ROOT+TEST_LOCATIONS_FILE, sep=';', names=['x','y'], dtype=float)

train_features = df_train_strengths
train_target = df_train_locs
generated_train_features = df_train_strengths
generated_train_target = df_train_locs
test_features = df_test_strengths
test_target = df_test_locs

normalization_values = np.array(train_features)

### Define all models
Currently:
- 3 hidden layer mixed with dropout layers (basic_model) (deep-learn-paper)

In [4]:
def gen_basic_model():
    normalizer = layers.Normalization()
    normalizer.adapt(normalization_values)

    model = tf.keras.models.Sequential([
      normalizer,
      layers.Dense(500, activation='relu'),
      layers.Dropout(0.5),
      layers.Dense(500, activation='relu'),
      layers.Dropout(0.5),
      layers.Dense(2, activation='relu'),
      layers.Dropout(0.5)
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
    loss_function = tf.keras.losses.MeanSquaredError()

    model.compile(optimizer=optimizer,
                    loss=loss_function)
    
    return model


In [5]:
# Define more models

### Train and evaluate all models in the same way

In [6]:
model_generators = [
    (gen_basic_model, 'Basic model, 3 hidden layers (500) with dropout (0.5)')
]
EPOCHS = 30000
BATCH_SIZE = 8


models_default = [m[0]() for m in model_generators]
models_generated = [m[0]() for m in model_generators]

histories = []
es = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2000)
for i, model in enumerate(models_default):
    print(f"Training model '{model_generators[i][1]}' on original data")
    histories.append(model.fit(train_features, train_target, epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=[es]))

for i, model in enumerate(models_generated):
    print(f"Training model '{model_generators[i][1]}' on generated data")
    histories.append(model.fit(train_features, train_target, epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=[es]))

Training model 'Basic model, 3 hidden layers (500) with dropout (0.5)' on original data
Epoch 1/30000


2022-12-31 16:30:38.651941: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2022-12-31 16:30:38.673923: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f3d110a7ca0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-12-31 16:30:38.673939: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2022-12-31 16:30:38.677475: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2022-12-31 16:30:38.677727: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-12-31 16:30:38.755573: I tensorflow/compiler/jit/xla_compilation_cache.cc:47

16/16 [==============================] - 1s 1ms/step - loss: 663.8784
Epoch 2/30000
16/16 [==============================] - 0s 1ms/step - loss: 639.1157
Epoch 3/30000
16/16 [==============================] - 0s 1ms/step - loss: 615.5715
Epoch 4/30000
16/16 [==============================] - 0s 1ms/step - loss: 595.2923
Epoch 5/30000
16/16 [==============================] - 0s 1ms/step - loss: 563.2704
Epoch 6/30000
16/16 [==============================] - 0s 2ms/step - loss: 522.6745
Epoch 7/30000
16/16 [==============================] - 0s 2ms/step - loss: 509.3668
Epoch 8/30000
16/16 [==============================] - 0s 1ms/step - loss: 468.9121
Epoch 9/30000
16/16 [==============================] - 0s 2ms/step - loss: 457.0352
Epoch 10/30000
16/16 [==============================] - 0s 1ms/step - loss: 424.3879
Epoch 11/30000
16/16 [==============================] - 0s 2ms/step - loss: 438.9975
Epoch 12/30000
16/16 [==============================] - 0s 2ms/step - loss: 418.5944
Epo

In [7]:
for i, model in enumerate(models_default):
    print(f"Evaluating original data model '{model_generators[i][1]}'")
    model.evaluate(test_features, test_target)

for i, model in enumerate(models_generated):
    print(f"Evaluating generated data model '{model_generators[i][1]}'")
    model.evaluate(test_features, test_target)

Evaluating original data model 'Basic model, 3 hidden layers (500) with dropout (0.5)'
15/15 [==============================] - 0s 831us/step - loss: 218.6404
Evaluating generated data model 'Basic model, 3 hidden layers (500) with dropout (0.5)'
15/15 [==============================] - 0s 907us/step - loss: 210.2543
